In [1]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="mistral")


In [4]:
llm.invoke("what model are you")

" I am a model of the Mistral AI's family of models, specifically trained as a conversational agent."

In [7]:
from langchain_ollama import OllamaLLM
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage
from langgraph.graph import StateGraph, END

In [8]:
llm = OllamaLLM(model="mistral")

In [15]:
def get_weather(location):
    return f"The weather in {location} is sunny and 25\u00b0C."

In [16]:
def get_time(location):
    return f"The time in {location} is 3 PM."

In [17]:
tools = {
    "get_weather":get_weather,
    "get_time":get_time
}

In [9]:
def define_state():
    return {"messages":[]}

In [10]:
def agent_node(state):
    messages = state["messages"]
    response = llm.invoke(messages)
    state["messages"].append(AIMessage(content=response))
    return state

In [60]:
system_prompt = """
    You are a helpful AI assistant with access to the following tools:

    - get_weather: Use this when the user asks about the weather.
    - get_time: Use this when the user asks for the current time.

    Your rules:
    1. If a user asks something related to time or weather, DO NOT respond directly.
    2. Instead, respond using this exact format:

    USE TOOL: tool_name, {"arg_name": "arg_value"}

    Example:

    User: What is the weather in Mumbai?
    Assistant: USE TOOL: get_weather, {"location": "Mumbai"}

    User: What's the time in Tokyo?
    Assistant: USE TOOL: get_time, {"location": "Tokyo"}

    3. Do not quote location names in natural language responses.
    4. If the question is unrelated to tools, respond normally.


    Important:( Strictly follow this)
    - DO NOT add parentheses or explanations after the tool call.
    - DO NOT generate tool output yourself.
    - DO NOT repeat the tool call later again.

"""

In [61]:
msg = agent_node({"messages" : [SystemMessage(content=system_prompt),HumanMessage(content="what is time in paris")]})

In [62]:
import os,json

In [63]:
def tool_node(state):
    last_msg = state["messages"][-1].content

    import re
    match = re.search(r'USE TOOL:\s*(\w+),\s*(\{.*\})', last_msg)
    if match:
        tool_name = match.group(1)
        args = json.loads(match.group(2))
        if tool_name in list(tools.keys()):
            resp = tools[tool_name](args["location"])
            tool_mesg = ToolMessage(content=resp, tool_call_id=f"{tool_name}-1")
            state["messages"].append(tool_mesg)
    return state
    


In [64]:
new_msg = tool_node(msg)

In [65]:
agent_node(new_msg)

{'messages': [SystemMessage(content='\n    You are a helpful AI assistant with access to the following tools:\n\n    - get_weather: Use this when the user asks about the weather.\n    - get_time: Use this when the user asks for the current time.\n\n    Your rules:\n    1. If a user asks something related to time or weather, DO NOT respond directly.\n    2. Instead, respond using this exact format:\n\n    USE TOOL: tool_name, {"arg_name": "arg_value"}\n\n    Example:\n\n    User: What is the weather in Mumbai?\n    Assistant: USE TOOL: get_weather, {"location": "Mumbai"}\n\n    User: What\'s the time in Tokyo?\n    Assistant: USE TOOL: get_time, {"location": "Tokyo"}\n\n    3. Do not quote location names in natural language responses.\n    4. If the question is unrelated to tools, respond normally.\n\n\n    Important:( Strictly follow this)\n    - DO NOT add parentheses or explanations after the tool call.\n    - DO NOT generate tool output yourself.\n    - DO NOT repeat the tool call l